In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 40.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12=

# 初始化学生模型

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer


teacher_model = AutoModelForSequenceClassification.from_pretrained("/kaggle/input/bert-base-teacher")

# 学生模型 BERT-Tiny
student_model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny")

tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny") 

if hasattr(student_model, "gradient_checkpointing_disable"):
    student_model.gradient_checkpointing_disable()
else:
    student_model.config.gradient_checkpointing = False

2025-11-26 15:21:50.144747: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764170510.515151      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764170510.718737      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

# 加载数据

In [3]:
import pandas as pd
import datasets
from sklearn.model_selection import train_test_split
from transformers import DataCollatorWithPadding

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, max_length=512)

# 加载数据
train = pd.read_csv("/kaggle/input/corpus-imdb/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("/kaggle/input/corpus-imdb/testData.tsv", header=0, delimiter="\t", quoting=3)
train, val = train_test_split(train, test_size=.2)
    
train_dict = {'labels': train["sentiment"], 'text': train['review']}
val_dict = {'labels': val["sentiment"], 'text': val['review']}
test_dict = {"text": test['review']}

train_dataset = datasets.Dataset.from_dict(train_dict)
val_dataset = datasets.Dataset.from_dict(val_dict)
test_dataset = datasets.Dataset.from_dict(test_dict)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

# 蒸馏训练

In [4]:
from transformers import Trainer, TrainingArguments

class DistillationTrainingArguments(TrainingArguments): 
    def __init__(self, *args, alpha=0.5, temperature=2.0, **kwargs): 
        super().__init__(*args, **kwargs) 
        self.alpha = alpha 
        self.temperature = temperature 
        
class DistillationTrainer(Trainer): 
    def __init__(self, *args, teacher_model=None, **kwargs): 
        super().__init__(*args, **kwargs) 
        self.teacher = teacher_model 
        # place teacher on same device as student
        self._move_model_to_device(self.teacher, self.model.device) 
        self.teacher.eval() 
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None) : 
        labels = inputs.pop("labels")
        # compute student output
        outputs_student = model(**inputs)

        # compute teacher output
        with torch.no_grad(): 
            outputs_teacher = self.teacher(**inputs) 
            
        # assert size
        assert outputs_student.logits.size() == outputs_teacher.logits.size()
        
        # 计算硬标签损失
        student_loss = F.cross_entropy(outputs_student.logits, labels)
        
        # Soften probabilities and compute distillation loss
        # 计算 KL 散度
        loss_function = nn.KLDivLoss(reduction="batchmean") 
        
        # 计算学生模型和教师网络数值输出的交叉熵损失
        loss_logits = (loss_function(
            # logits首先除以 temperature, 以增加对错误分类的关注
            F.log_softmax(outputs_student.logits / self.args.temperature, dim=-1), 
            F.softmax(outputs_teacher.logits / self.args.temperature, dim=-1)) * (self.args.temperature ** 2)) 
        
        # Return weighted student loss
        loss = self.args.alpha * student_loss + (1. - self.args.alpha) * loss_logits 
        return (loss, outputs_student) if return_outputs else loss


# 模型训练

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

os.makedirs('/kaggle/working/checkpoint', exist_ok=True)
os.makedirs('/kaggle/working/logs', exist_ok=True)
training_args = DistillationTrainingArguments( 
    output_dir="/kaggle/working/checkpoint", 
    num_train_epochs=3, 
    per_device_train_batch_size=2, 
    per_device_eval_batch_size=4, 
    gradient_accumulation_steps=8, 
    gradient_checkpointing=False, 
    warmup_steps=500,  
    # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  
    # strength of weight decay
    logging_dir='/kaggle/working/logs',  
    # directory for storing logs
    logging_steps=100, 
    save_strategy="no", 
    eval_strategy="epoch", 
    # distilation parameters
    alpha=0.5, 
    temperature=4.0,
    report_to="none"
) 
     
trainer = DistillationTrainer( 
    model=student_model, 
    args=training_args, 
    teacher_model=teacher_model, 
    train_dataset=tokenized_train,  # training dataset
    eval_dataset=tokenized_val,  # evaluation dataset
    processing_class=tokenizer, 
    data_collator=data_collator, 
    compute_metrics=compute_metrics, 
) 
trainer.train() 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.656600,1.430904,0.826800
2,1.104500,0.960790,0.868000
3,0.836300,0.930372,0.875400


TrainOutput(global_step=1875, training_loss=1.3805338256835937, metrics={'train_runtime': 2175.082, 'train_samples_per_second': 27.585, 'train_steps_per_second': 0.862, 'total_flos': 64038720588960.0, 'train_loss': 1.3805338256835937, 'epoch': 3.0})

In [6]:
prediction_outputs = trainer.predict(tokenized_test) 
test_pred = np.argmax(prediction_outputs[0], axis=-1).flatten() 
print(test_pred) 
result_output = pd.DataFrame(data={"id": test["id"], "sentiment": test_pred}) 
result_output.to_csv("/kaggle/working/tiny_bert_student.csv", index=False, quoting=3) 

[1 0 1 ... 0 1 0]
